## 文档说明
本文档用以计算Kistler 游泳出发测力台的力值数据

Kistler(KiSwim) 游泳出发测力台原始导出的数据只有电压值

根据工程师的说明讲力值转换为力值

## 步骤
1. 读取每一列的电压数据
2. 将每个传感器的数据转换为力值
3. 对力值数据进行滤波
4. 将传感器的力值进行叠加
5. 将传感器的力值转换为水平力值以及垂直力值
6. 将水平力值和垂直力值进行叠加分别计算前、后台合力
7. 计算前、后台以及抓手在水平和垂直方向合力

## Todo
增加判断

In [42]:
## 导入包
import os
import pandas as pd
import numpy as np 
import scipy.signal as signal
import matplotlib.pyplot as plt
import datetime

In [34]:
## 定义路径
Voltage_dir = "/Users/wangshuaibo/Documents/ScriptsofShuai/Notes/ExampleData/KiSwim/电压数据"
Forece_dir =  "/Users/wangshuaibo/Documents/ScriptsofShuai/Notes/ExampleData/KiSwim/力值数据"

## 根据电压计算每个传感器的原始力值
Force [N]  = ((Force [Volts] – Force[V] offset)/sensitivity)*1000

传感器灵敏度，可以从 Kpass 软件下的 KPpssSetup.csv 中获取

In [35]:
# 定义前台和水平面的夹角
angle = 10.2 # 每块测力台的数值不同，从测力台的 pdf 当中获取
theta_rad = np.deg2rad(angle) 
## 定义不同传感器的灵敏度(sensitivity)
sensitivity = {
    '抓手Z方向电压': 3.899,
    '抓手X方向电压': 1.842,
    
    '前台Y14电压': 3.861,
    '前台Y23电压': 3.861,

    '前台Z1电压': 3.611,
    '前台Z2电压': 3.611,
    '前台Z3电压': 3.611,
    '前台Z4电压': 3.611,

    "N/A_1": 1, # 1 为填补空缺，不用修改
    "N/A_2": 1, # 1 为填补空缺，不用修改

    '后台Y14电压': 3.864,
    '后台Y23电压': 3.864,
    
    '后台Z1电压': 3.657,
    '后台Z2电压': 3.657,
    '后台Z3电压': 3.657,
    '后台Z4电压': 3.657
}

In [36]:
# 按照Kistler 工程师邮件中的说明，命名
head_list = ["抓手Z方向电压","抓手X方向电压",
             "前台Y14电压","前台Y23电压",
             "前台Z1电压","前台Z2电压","前台Z3电压","前台Z4电压",
             "N/A_1","N/A_2",
             "后台Y14电压","后台Y23电压",
             "后台Z1电压","后台Z2电压","后台Z3电压","后台Z4电压"]
usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

In [47]:
for filename in os.listdir(Voltage_dir):
    if filename.endswith(".csv"):
        # 读取文件
        Voltage_path = os.path.join(Voltage_dir, filename) # 电压数据路径
        offset_data = pd.read_csv(Voltage_path, skiprows=4, nrows=1,usecols=usecols, names=head_list) # 读取偏移量
        voltage_data = pd.read_csv(Voltage_path, skiprows=6, usecols=usecols, names=head_list) # 读取电压数据

        # 计算各个传感器的电压值
        # 减去偏移量并将结果汇总到新的列，使用 pandas 广播功能
        voltage_data_adjusted = voltage_data.sub(offset_data.iloc[0].T)
        # 根据调整后的电压计算力值，使用 pandas 广播功能
        Sen_Force = voltage_data_adjusted / pd.Series(sensitivity) * 1000
        # 除使用广播的方式进行计算，还可以使用 apply 函数
        #Sen_Force_apply = voltage_data_adjusted.apply(lambda x: x/ sensitivity[x.name] * 1000)
        
        # 计算前后台的水平力和垂直力
        # 先计算相对于测力台本身的数据
        Plate_Force = pd.DataFrame()
        Plate_Force['前台传感器水平力'] = Sen_Force['前台Y14电压'] + Sen_Force['前台Y23电压']
        Plate_Force["前台传感器垂直力"] = Sen_Force['前台Z1电压'] + Sen_Force['前台Z2电压'] + Sen_Force['前台Z3电压'] + Sen_Force['前台Z4电压']
        # 将传感器的力值转化到水平和垂直方向
        Plate_Force['前台水平力'] =  Plate_Force["前台传感器垂直力"] * np.sin(theta_rad) + Plate_Force['前台传感器水平力'] * np.cos(theta_rad)
        Plate_Force['前台垂直力'] =  Plate_Force["前台传感器垂直力"] * np.cos(theta_rad) - Plate_Force['前台传感器水平力'] * np.sin(theta_rad)
        Plate_Force["前台合力"] = (Plate_Force["前台水平力"]**2 + Plate_Force["前台垂直力"]**2)**0.5

        Plate_Force["后台传感器水平力"] = Sen_Force['后台Y14电压'] + Sen_Force['后台Y23电压']
        Plate_Force['后台传感器垂直力'] = Sen_Force['后台Z1电压'] + Sen_Force['后台Z2电压'] + Sen_Force['后台Z3电压'] + Sen_Force['后台Z4电压']
        Plate_Force['后台水平力'] =  Plate_Force["后台传感器垂直力"] * np.sin(theta_rad) + Plate_Force['后台传感器水平力'] * np.cos(theta_rad)
        Plate_Force['后台垂直力'] =  Plate_Force["后台传感器垂直力"] * np.cos(theta_rad) - Plate_Force['后台传感器水平力'] * np.sin(theta_rad)
        Plate_Force["后台合力"] = (Plate_Force["后台水平力"]**2 + Plate_Force["后台垂直力"]**2)**0.5

        Plate_Force["抓手传感器水平力"] = Sen_Force['抓手X方向电压'] # Gx 为平行于水平方向
        Plate_Force["抓手传感器垂直力"] = Sen_Force['抓手Z方向电压']  # Gz 为垂直于水平方向
        Plate_Force["抓手水平力"] = Plate_Force["抓手传感器水平力"]
        Plate_Force["抓手垂直力"] = Plate_Force["抓手传感器垂直力"] * -1

        # 此处计算前后台在水平方面和垂直方向的合力
        Plate_Force["传感器方向水平合力"] = Plate_Force['前台传感器水平力'] + Plate_Force['后台传感器水平力']
        Plate_Force["传感器方向垂直合力"] = Plate_Force['前台传感器垂直力'] + Plate_Force['后台传感器垂直力']
        
        # 将测力台垂直和水平力分解到大地水平方向和垂直方向
        Plate_Force["水平力"] = Plate_Force["传感器方向垂直合力"] * np.sin(theta_rad) + Plate_Force["传感器方向水平合力"] * np.cos(theta_rad) + Plate_Force["抓手传感器水平力"]
        Plate_Force["垂直力"] = Plate_Force["传感器方向垂直合力"] * np.cos(theta_rad) - Plate_Force["传感器方向水平合力"] * np.sin(theta_rad) + Plate_Force["抓手传感器垂直力"]

        ## 对力值信号进行滤波
        # 在根据测力台信号计算垂直和水平力之前，使用 10HZ 的低通滤波对测量信号进行数字平滑处理。
        # ref：Mason, Bruce, Alison Alcock和John Fowlie. 《A Kinetic Analysis and Recommendations for Elite Swimmers Performing the Sprint Start》, 4, 2007.
        # 武哥是在计算得到水平力之后进行滤波，此处与武哥保持一致，两种滤波方式相差不大
        target_columns = ['前台合力','前台水平力','前台垂直力',
            '后台合力','后台水平力','后台垂直力',
            '抓手垂直力','抓手水平力',
            '垂直力','水平力']

        fs = 500 # 测力台采集频率为 500 Hz
        n = 2 # 滤波器阶数
        Wn = 10 / (fs / 2) # 计算归一化频率， 截止频率为  10 Hz
        b, a = signal.butter(n, Wn, 'low') # 低通滤波器
        # 对选定的列进行滤波
        filtered_data = Plate_Force[target_columns].apply(lambda column: signal.filtfilt(b, a, column))

        # 除以体重，对力值进行归一化处理

        # 计算运动员体重(单位:N)
        # 运动员静止站立时垂直
        # 测力台会在发令枪响前 500 整数开始采集数据
        weight_time = 500-1 # 枪响枪前会采集 500 个数据，这里默认为 500
        Weight_N = Plate_Force.loc[0:weight_time,"垂直力"].mean()
        normalized_data = filtered_data / Weight_N

        # 将数据写入文件
        current_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S_")
        excel_filename = current_time +  filename.replace(".csv", ".xlsx")
        excel_path = os.path.join(Forece_dir, excel_filename)
        normalized_data.to_excel(excel_path, index=False)

        print(f"{filename}已经完成处理")

print("所有文件已经处理完成")
print("处理后的文件保存在：", Forece_dir)

杨金潼.csv
王曦喆蝶泳.csv
汪顺.csv
陈俊儿.csv
董洁.csv
王长浩蝶泳.csv
董志豪.csv
王浩宇.csv
牛广盛.csv
刘培鑫.csv
张展硕.csv
潘展乐.csv
季新杰.csv
覃海洋.csv
王曦喆自由泳.csv
洪金权.csv
王长浩自由泳.csv
纪奕存.csv
